# Pre-Processing

Raw Collect Earth data is a hot mess. This notebook will clean and prepare the data for downstream analysis

In [1]:
import pandas as pd
import numpy as np

## Import csv and clean

In [2]:
data='data/raw/ceo-RCMRD--Cross-Validation-point---Project-3-sample-data-2021-03-09.csv'
outname='data/clean/RCMRD_CV_Project-3.csv'

In [3]:
# Read in the validation data csv
df = pd.read_csv(data, delimiter=",").drop(['sample_id',
                                            'imagery_title',
                                            'collection_time',
                                            'analysis_duration',
                                            'pl_plotid',
                                            'imagery_attributions',
                                            'email'], axis=1).rename(columns={'What is the feature?':'class',
                        'Enter Months[1-12] in 2018, water was observed?':'water',
                        'Enter Months[1-12] in 2018, water was not observed?':'no_water',
                        'Enter Months[1-12] in 2018, image was bad?':'bad_image',
                        'Enter months[1-12] in 2018, that you are unsure if you observe water or not? ':'not_sure'})

#Converting column type to string if not already
df['not_sure'] = df['not_sure'].astype(str)
df['water'] = df['water'].astype(str)
df['no_water'] = df['no_water'].astype(str)
df['bad_image'] = df['bad_image'].astype(str)

#Making sure that the columns for each label class is string for further analysis  
cols = ['water','no_water','bad_image','not_sure']
for col in cols:
    df[col] = df[col].str.replace('[','')
    df[col] = df[col].str.replace(']','')
    df[col] = df[col].str.replace('&','')
    df[col] = [''.join(c.split()) for c in df[col]]

#replacing the name of months with their numerical values
replacements = {'water': {r'Jan':'1', r'Feb':'2',r'Mar':'3',r'Apr':'4',r'May':'5',r'Jun':'6',r'Jul':'7',r'Aug':'8',r'Sep':'9',r'Oct':'10',r'Nov':'11',r'Dec':'12'},
               'no_water': {r'Jan':'1', r'Feb':'2',r'Mar':'3',r'Apr':'4',r'May':'5',r'Jun':'6',r'Jul':'7',r'Aug':'8',r'Sep':'9',r'Oct':'10',r'Nov':'11',r'Dec':'12'},
               'bad_image':{r'Jan':'1', r'Feb':'2',r'Mar':'3',r'Apr':'4',r'May':'5',r'Jun':'6',r'Jul':'7',r'Aug':'8',r'Sep':'9',r'Oct':'10',r'Nov':'11',r'Dec':'12'}}

df.replace(replacements, regex=True, inplace=True)
df.head()

,plot_id,lon,lat,flagged,analyses,sample_geom,water,no_water,bad_image,not_sure,class,Comment
0,137712516,13.880937,7.103984,False,1,POINT(13.88093723 7.103983557),0,1-12,0,0.0,Herbaceous/Grassland,Na
1,137712517,18.989746,-16.394333,False,1,POINT(18.98974647 -16.39433274),1-12,0,0,0.0,Open water - freshwater,A river
2,137712518,30.271039,-11.558283,False,1,POINT(30.27103949 -11.55828322),1-12,0,"1,2,12",0.0,Open water - freshwater,months nothing observed due to clouds
3,137712519,13.426054,-19.597793,False,1,POINT(13.42605391 -19.59779325),0,1-12,1,0.0,Barren (Bare soil / Rocky Land),Na
4,137712520,29.304374,-22.439381,False,1,POINT(29.30437356 -22.43938067),0,1-12,31,0.0,Barren (Bare soil / Rocky Land),Open pit Mining


## Split strings in each class

Columns identified as no water, water, bad image, not sure. These classes will be assigned a value as 0, 1, 2, 3 respectively.


In [4]:
def update_table(column_str, row, newtable):
    
    monthstr=row[column_str]
    if monthstr!='0'and monthstr!='nan':
        monthlist=[[int(i) for i in s.split('-')] for s in monthstr.split(',')]

        for l in monthlist:
            if len(l)==1: l=[l[0],l[0]]
            for i in range(l[0], l[1]+1):
                newrow=row[['plot_id','lon','lat','flagged','analyses','water',
                            'no_water','bad_image','not_sure','class','Comment']]
                newrow['month']=f'{i:02d}'
                if column_str=='no_water':
                    newrow['waterflag']='0'
                if column_str=='water':
                    newrow['waterflag']='1'
                if column_str=='bad_image':
                    newrow['waterflag']='2'
                if column_str=='not_sure':
                    newrow['waterflag']='3'
                newrow["s2_year"]='2018'
                newtable=newtable.append(newrow)
    
    return newtable

def split_str(row, newtable):
    try:
        newtable = update_table('no_water', row, newtable)
        newtable = update_table('water', row, newtable)
        newtable = update_table('bad_image', row, newtable)
        newtable = update_table('not_sure', row, newtable)
    except ValueError:
        pass
    
    return newtable

In [5]:
#Making an empty dataframe
result = pd.DataFrame()
for irow in range(len(df)):
    try:
        result=split_str(df.iloc[irow], result)
        result.update(result)
    except ValueError:
        continue
    

In [6]:
len(result)

1009

In [7]:
#remove duplicates
result=result.drop_duplicates(subset=['lat','lon','month','plot_id'], keep=False)
len(result)

861

In [8]:
result=result[['lat','lon','month','plot_id','class','waterflag']].reset_index(drop=True)

In [9]:
result

,lat,lon,month,plot_id,class,waterflag
0,7.103984,13.880937,01,137712516.0,Herbaceous/Grassland,0
1,7.103984,13.880937,02,137712516.0,Herbaceous/Grassland,0
2,7.103984,13.880937,03,137712516.0,Herbaceous/Grassland,0
3,7.103984,13.880937,04,137712516.0,Herbaceous/Grassland,0
4,7.103984,13.880937,05,137712516.0,Herbaceous/Grassland,0
...,...,...,...,...,...,...
856,-5.424299,29.747131,06,137712595.0,Open water - freshwater,1
857,-5.424299,29.747131,07,137712595.0,Open water - freshwater,1
858,-5.424299,29.747131,08,137712595.0,Open water - freshwater,1
859,-5.424299,29.747131,09,137712595.0,Open water - freshwater,1


In [10]:
result.to_csv(outname)